In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.expected_conditions import presence_of_element_located
from webdriver_manager.chrome import ChromeDriverManager 
import pandas as pd
from time import sleep

# Search Parameters

In [ ]:
range_low = '35078000'
range_high = '35080000'
start_date = '01/08/2020'
end_date = '31/08/2020'

# Input Desired Parameters

In [ ]:
driver = webdriver.Chrome(ChromeDriverManager().install())

driver.get('https://www.hacienda.go.cr/tica/web/hcitrncm1.aspx')

# Dealing with Captcha and Filling Form Data

In [ ]:
img_src = driver.find_element_by_xpath('//div[@id="captchaImage"]/img')

In [ ]:
img_src = img_src.get_attribute('src')

In [ ]:
captcha_number = int(img_src.split('/')[-1].split('.')[0])
captcha_number

In [ ]:
captchas = pd.read_csv('../Captchas/captchas.csv', index_col=0)

In [ ]:
captcha_value = captchas.iloc[captcha_number - 1]

In [ ]:
captcha_value

In [ ]:
driver.find_element_by_xpath('//select[@id="vVADUANA"]/option[@value="X"]').click()

In [ ]:
date_i = driver.find_element_by_xpath('//div[@id="vVFCHIN_dp_container"]/input')

In [ ]:
date_i.send_keys(start_date)

In [ ]:
date_f = driver.find_element_by_xpath('//div[@id="vVFCHFN_dp_container"]/input')

In [ ]:
date_f.send_keys(end_date)

In [ ]:
range_start = driver.find_element_by_xpath('//td/input[@id="vVNCMINI"]')
range_start.clear()
range_start.send_keys(range_low)

In [ ]:
range_end = driver.find_element_by_xpath('//td/input[@id="vVNCMFIN"]')
range_end.clear()
range_end.send_keys(range_high)

In [ ]:
captcha_field = driver.find_element_by_xpath('//input[@id="_cfield"]')

In [ ]:
captcha_field.clear()

In [ ]:
captcha_field.send_keys(captcha_value)

In [ ]:
driver.find_element_by_xpath('//input[@name="BUTTON1"]').click()


# Obtaining Data

## Main iFrame

In [ ]:
source = driver.page_source

In [ ]:
tables = pd.read_html(source)

In [ ]:
main_pg_data = tables[9]

In [ ]:
duas_df = main_pg_data.groupby('Numero').size()

In [ ]:
duas_df

In [ ]:
duas_df.iloc[:1].sum()

In [ ]:
for i in range(1, len(duas_df) + 1):
    print(i)

In [ ]:
xpath = '//table[@id="SubfileContainerTbl"]/tbody/tr/td[@colindex="2"]/span[@id="span_vVNUMECORRE_0001"]/a'

In [ ]:
driver.find_element_by_xpath(xpath).click()
sleep(1)

In [ ]:
iframe = driver.find_element_by_xpath('//iframe')

In [ ]:
driver.switch_to.frame(iframe)

In [ ]:
declarante = driver.find_element_by_xpath('//span[@id="span_vVNOMDECL"]').text

In [ ]:
tipo_cambio_c = float(driver.find_element_by_xpath('//span[@id="span_TIPCAMCOM"]').text)

In [ ]:
tipo_cambio_v = float(driver.find_element_by_xpath('//span[@id="span_TIPCAMVEN"]').text)

In [ ]:
n_lineas = int(driver.find_element_by_xpath('//span[@id="span_vVLINEAS"]').text)

## Lineas DUA iFrame

In [ ]:
driver.find_element_by_xpath('//input[@name="LINEAS"]').click()

In [ ]:
dfs = []
pgs = n_lineas // 20 + 1

In [ ]:
for page in range(pgs):
    
    sleep(1)
    
    lineas_src = driver.page_source
    dfs.append(pd.read_html(lineas_src))
    
    driver.find_element_by_xpath('//input[@name="SIGUIENTE"]').click()
    
    

In [ ]:
dfs = [dfs[i][1] for i in range(pgs)]

In [ ]:
lineas_df = pd.concat(dfs)

In [ ]:
lineas_df.head()

In [ ]:
lineas_df[['Linea', 'Partida', 'Bultos']].astype('int').dtypes

In [ ]:
lineas_df['Partida'] = lineas_df['Partida'].apply(lambda x: int(x/10000))

In [ ]:
lineas_df = lineas_df[(lineas_df['Partida'] > int(range_low)) & (lineas_df['Partida'] < int(range_high))]

In [ ]:
n_bultos = lineas_df['Bultos'].sum()

In [ ]:
ActionChains(driver).send_keys(Keys.ESCAPE).perform()

## Manifesto/Stock iFrame

Solo hace falta ir popr la razon social, en facturas esta el peso y el costo unitario para calcular el precio

In [ ]:
driver.find_element_by_xpath('//input[@name="AER"]').click()

In [ ]:
driver.find_element_by_xpath('//span[@id="span_CGMOVSKID_0001"]').click()

In [ ]:
kg = float(driver.find_element_by_xpath('//span[@id="span_CGMOVKGS_0001"]').text)

In [ ]:
razon_social = driver.find_element_by_xpath('//span[@id="span_vVCONSNOM_0001"]').text

In [ ]:
ActionChains(driver).send_keys(Keys.ESCAPE).perform()

## Facturas iFrame

In [ ]:
ActionChains(driver).send_keys(Keys.ESCAPE).perform()

In [ ]:
driver.find_element_by_xpath('//input[@name="FACTURAS"]').click()

In [ ]:
fact_src = driver.page_source

In [ ]:
fact_dfs = pd.read_html(fact_src)

In [ ]:
lines_of_interest = lineas_df['Linea'].values

In [ ]:
fact_dfs = fact_dfs[4]

In [ ]:
fact_dfs['Lin.Dua'].astype('int').dtypes

In [ ]:
fact_dfs[['Cant. prod. Und comerc', 'Precio unid. mercancía']].astype('float').dtypes

In [ ]:
fact_dfs = fact_dfs.set_index('Lin.Dua').loc[lines_of_interest]

In [ ]:
driver.quit()

